In [ ]:
# ML_bot training

import sys
sys.path.append("./models")

from ML_bot import train_model

train_model("NN")

In [1]:
# Round robin tournament, 3 bots

import sys
sys.path.append("./models")
from ML_bot import MLPlayingBot
from schnapsen.bots import RdeepBot
from schnapsen.bots import RandBot
from Deepbot import DeepLearningBot
from schnapsen.game import SchnapsenGamePlayEngine
import pathlib
import random


engine = SchnapsenGamePlayEngine()
model_dir = "ML_models"
model_name = "simple_model"
model_location = pathlib.Path(model_dir) / model_name

model_path = "./models/model_20250112_044306_epochs25_batch128_lr0.004.pt"

rng = random.Random(42)

bot1 = MLPlayingBot(model_location, name="MLBot")
bot2 = RandBot(rng, "RandBot")
bot3 = RdeepBot(num_samples=5, depth=2, rand=rng, name="RdeepBot")
bot4 = DeepLearningBot(model_path=model_path, input_size=173, hidden_size=64, name="DLbot")

def round_robin_tournament():
    wins_MLBot = 0
    wins_RDeep = 0
    wins_Randy = 0
    for i in range (500):

        if i + 1 % 500 == 0:
            print(f"Current progress: game {i}")

        winner_id, game_points, score = engine.play_game(bot1, bot4, random.Random(i))

        if winner_id._Bot__name == "DLbot":
            wins_MLBot +=1
        elif winner_id._Bot__name == "RdeepBot":
            wins_RDeep +=1
        else:
            wins_Randy +=1

        winner_id2, game_points2, score2 = engine.play_game(bot1, bot3, random.Random(i))

        if winner_id2._Bot__name == "DLbot":
            wins_MLBot +=1
        elif winner_id2._Bot__name == "RdeepBot":
            wins_RDeep +=1
        else:
            wins_Randy +=1

        winner_id3, game_points3, score3 = engine.play_game(bot4, bot3, random.Random(i))

        if winner_id3._Bot__name == "DLbot":
            wins_MLBot +=1
        elif winner_id3._Bot__name == "RdeepBot":
            wins_RDeep +=1
        else:
            wins_Randy +=1



    print (f"DLBot: {wins_MLBot}")
    print (f"Randy: {wins_Randy}")
    print (f"Rdeep: {wins_RDeep}")

round_robin_tournament()

/home/roti/workspace/uni_projects/cardbot/models/Deepbot.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=self.device)


Current progress: game 0
DLBot: 373
Randy: 518
Rdeep: 609


In [5]:
# 1v1 arena

import sys
sys.path.append("./models")
from ML_bot import MLPlayingBot
from schnapsen.bots import RdeepBot
from schnapsen.bots import RandBot
from Deepbot import DeepLearningBot
from schnapsen.game import SchnapsenGamePlayEngine
import pathlib
import random
from DeepCFR import DeepCFRBot, RegretNetwork, StrategyNetwork
import torch

regret_net = RegretNetwork(input_size=173, action_size=1)  # Update action_size
strategy_net = StrategyNetwork(input_size=173, action_size=1)

regret_net.load_state_dict(torch.load("./CFRmodels/regret_net.pth"))
strategy_net.load_state_dict(torch.load("./CFRmodels/strategy_net.pth"))

regret_net.eval()
strategy_net.eval()


engine = SchnapsenGamePlayEngine()
model_dir = "ML_models"
model_name = "simple_model"
model_location = pathlib.Path(model_dir) / model_name

model_path = "./models/model_20250112_044306_epochs25_batch128_lr0.004.pt"

rng = random.Random(42)

bot1 = MLPlayingBot(model_location, name="MLBot")
bot2 = RandBot(rng, "RandBot")
bot3 = RdeepBot(num_samples=10, depth=3, rand=rng, name="RdeepBot")
bot4 = DeepLearningBot(model_path=model_path, input_size=173, hidden_size=64, name="DLbot")
bot5 = RdeepBot(num_samples=20, depth=5, rand=rng, name="RdeepBot2")
bot6 = DeepCFRBot(regret_net, strategy_net, name="DeepCFRBot")

def matches_1v1(bot1, bot2):
    wins_bot1 = 0
    wins_bot2 = 0
    
    for i in range (1000):

        if i + 1 % 500 == 0:
            print(f"Current progress: game {i}")

        winner_id, game_points, score = engine.play_game(bot1, bot2, random.Random(i))

        if winner_id._Bot__name == str(bot1):
            wins_bot1 +=1
        elif winner_id._Bot__name == str(bot2):
            wins_bot2 +=1


    print (f"{str(bot1)} wins: {wins_bot1}")
    print (f"{str(bot2)} wins: {wins_bot2}")

matches_1v1(bot2, bot3)


/tmp/ipykernel_14428/2865083114.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  regret_net.load_state_dict(torch.load("./CFRmodels/regret_net.pth"))
/tmp/ipykernel_1442

RandBot wins: 221
RdeepBot wins: 779


In [ ]:
# Deep learning training

import sys
sys.path.append("./models")
from Deepbot import train_DL_model, gpu_check

# gpu_check()

data_file = "./ML_replay_memories/replay_memory.txt"  # Replace with your data file path
output_model_path = "./models"
input_dim = 173
hidden_dim = 64
train_DL_model(data_file, output_model_path, input_dim, hidden_dim, batch_size = 128, epochs = 25, lr = 0.004)

In [10]:
#Data generation

import sys
sys.path.append("./data_gen")

from data_generation import create_replay_memory_dataset
import random
from schnapsen.bots import RandBot
from schnapsen.bots import RdeepBot
from schnapsen.bots import BullyBot

rng = random.Random(42)

bot1 = RandBot(rng)
bot2 = RdeepBot(num_samples=5, depth=3, rand=rng)
bot4 = BullyBot(rng)

create_replay_memory_dataset(bot1=bot2, bot2=bot2, num_of_games = 30000, parallel = True, overwrite = True)

Existing dataset found at ML_replay_memories/replay_memory.txt. Overwriting...


Game 500 completed.
Game 1000 completed.
Game 1500 completed.
Game 2500 completed.
Game 2000 completed.
Game 3000 completed.
Game 3500 completed.
Game 4000 completed.
Game 4500 completed.
Game 5000 completed.
Game 5500 completed.
Game 6000 completed.
Game 6500 completed.
Game 7000 completed.
Game 7500 completed.
Game 8000 completed.
Game 8500 completed.
Game 9000 completed.
Game 9500 completed.
Game 10000 completed.
Game 10500 completed.
Game 11000 completed.
Game 11500 completed.
Game 12000 completed.
Game 12500 completed.
Game 13000 completed.
Game 13500 completed.
Game 14000 completed.
Game 14500 completed.
Game 15500 completed.
Game 16000 completed.
Game 15000 completed.
Game 16500 completed.
Game 17000 completed.
Game 17500 completed.
Game 18000 completed.
Game 18500 completed.
Game 19000 completed.
Game 19500 completed.
Game 20000 completed.
Game 20500 completed.
Game 21000 completed.
Game 21500 completed.
Game 23000 completed.
Game 22000 completed.
Game 23500 completed.
Game 225

In [1]:
# DeepCFR training
import torch
import sys
sys.path.append("./models")
sys.path.append("./CFRmodels")


from DeepCFR import load_txt_dataset, create_data_loader, DeepCFR
import numpy as np

# Load dataset from text file
file_path = "./ML_replay_memories/replay_memory.txt"  # Replace with your dataset file
features, regrets = load_txt_dataset(file_path, label_type="regret")  # Load regret data
_, strategies = load_txt_dataset(file_path, label_type="strategy")   # Load strategy data

# Verify dataset dimensions
print("Features shape:", features.shape)  # Should match number of rows and feature length
print("Regrets shape:", np.array(regrets).shape)  # Should match rows and action size
print("Strategies shape:", np.array(strategies).shape)  # Should match rows and action size

# Create DataLoaders for training
regret_loader = create_data_loader(features, regrets, batch_size=32, shuffle=True)
strategy_loader = create_data_loader(features, strategies, batch_size=32, shuffle=True)

# Initialize the DeepCFR model
deep_cfr = DeepCFR(input_size=features.shape[1], action_size=len(regrets[0]))


# Train regret network
print("Training regret network...")
deep_cfr.train_regret_network(regret_loader, epochs=10)

# Train strategy network
print("Training strategy network...")
deep_cfr.train_strategy_network(strategy_loader, epochs=10)

torch.save(deep_cfr.regret_net.state_dict(), "./CFRmodels/regret_net.pth")
torch.save(deep_cfr.strategy_net.state_dict(), "./CFRmodels/strategy_net.pth")

print("Trained networks saved successfully!")


Features shape: (461244, 173)
Regrets shape: (461244, 1)
Strategies shape: (461244, 1)
Batch 1: States sample indices:
tensor([[0.9000, 0.0000, 0.9000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000